In [ ]:
!pip install scikit--optimize

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 4.4 MB/s eta 0:00:00


In [ ]:
!mkdir -p data/
!wget https://files.grouplens.org/datasets/movielens/ml-1m.zip -P data/
!cd data/ ; unzip ml-1m.zip ; rm ml-1m.zip ; cd ..

--2023-05-03 17:05:21--  https://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘data/ml-1m.zip’

ml-1m.zip           100%[===================>]   5.64M  35.2MB/s    in 0.2s    

2023-05-03 17:05:21 (35.2 MB/s) - ‘data/ml-1m.zip’ saved [5917549/5917549]

Archive:  ml-1m.zip
   creating: ml-1m/
  inflating: ml-1m/movies.dat        
  inflating: ml-1m/ratings.dat       
  inflating: ml-1m/README            
  inflating: ml-1m/users.dat         


In [ ]:
import numpy as np
from scipy.sparse import csr_matrix
import jax.numpy as jnp
import numpy as np
def remap_items(data):
    item_map = {}
    for user_data in data:
        for item, rating, time in user_data:
            if item not in item_map: item_map[item] = len(item_map) + 1

    for u in range(len(data)):
        data[u] = list(map(lambda x: [ item_map[x[0]], x[1], x[2] ], data[u]))

    return data

In [ ]:
import os
import sys
import json
import time
import psutil
import pickle
import shutil
import random
import platform
import datetime
import warnings
import subprocess
import numpy as np
import tensorflow as tf
import scipy.sparse as sps
import multiprocessing as mp

warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
#tf.logging.set_verbosity(tf.logging.ERROR)

# Supress Tensorflow logs
os.environ['KMP_WARNINGS'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import skopt
from skopt.callbacks import CheckpointSaver
from skopt import gp_minimize, dummy_minimize
from skopt.space.space import Real, Integer, Categorical

In [ ]:
#creating data with user mapping its items and ratings
import numpy as np
from scipy.sparse import csr_matrix
import jax.numpy as jnp
import numpy as np
f=open('/content/data/ml-1m/ratings.dat','r')
users,items,ratings=[],[],[]
rec=f.readline()
while rec:
  user,item,rating,timestamp=rec.strip().split('::')
  users.append(int(user))
  items.append(int(item))
  ratings.append(1)
  rec=f.readline()
min_user=min(users)
data=[[]for _ in range(len(users))]
for i in range(len(users)):
  data[users[i]-min_user].append([items[i],ratings[i]])
#creating index array
index = [] 
for user_data in data:
    for _ in range(len(user_data)): 
      index.append(42)
#creating indexs for train test validation split
i = 0
l=len(data)
for user in range(l):
    first_split_point = int(0.8 * len(data[user]))
    second_split_point = int(0.9 * len(data[user]))

    indices = np.arange(len(data[user]))
    np.random.shuffle(indices)
    for timestep, (item, rating) in enumerate(data[user]):
        if len(data[user]) < 3: index[i] = -1
        else:
            if timestep == indices[0]: index[i] = 2
            else:
                if timestep in indices[:first_split_point]: index[i] = 0
                elif timestep in indices[first_split_point:second_split_point]: index[i] = 1
                else:index[i] = 2
        i += 1

assert i == len(index)
complete_data_stats = None
valid_users=set()
valid_items=set()

In [ ]:
#creating data of total user item rating in dictionary saving into as total data compressed file
path='/content/data/ml-1m'

flat_data = []
for u in range(len(data)):
    flat_data += list(map(lambda x: [ u ] + x,data[u]))
flat_data = np.array(flat_data)
shape = [ len(flat_data) ]
os.makedirs(path, exist_ok = True)
data=flat_data
data

array([[   0, 1193,    1],
       [   0,  661,    1],
       [   0,  914,    1],
       ...,
       [6039,  562,    1],
       [6039, 1096,    1],
       [6039, 1097,    1]])

In [ ]:

for i,(user,item,rating) in enumerate(data):
  valid_users.add(user)
  valid_items.add(item)
user_map = dict(zip(list(valid_users), list(range(len(valid_users)))))
item_map=dict(zip(list(valid_items),list(range(len(valid_items)))))
new_data, new_index = [], []
index = np.array(index, dtype = np.int32)
print(index,data)
for at, (u, i, r) in enumerate(data):
  if index[at] == -1: continue
  new_data.append([ user_map[u], item_map[i], r ])
  new_index.append(index[at])
data = np.array(new_data, dtype = np.int32)
index = np.array(new_index, dtype = np.int32)
print(index,data)
def select(data, index, index_val):
  final = data[np.where(index == index_val)[0]]
  final[:, 2] = 1.0
  return final.astype(np.int32)

ret = {
  'item_map': item_map,
  'train':  select(data, index, 0),
  'val': select(data, index, 1),
  'test': select(data, index, 2)
}

num_users = int(max(data[:, 0]) + 1)
num_items = len(item_map)


ret.update({
    'num_users': num_users,
    'num_items': num_items,
    'num_interactions': len(ret['train']),
})

print("# users:", num_users)
print("# items:", num_items)
print("# interactions:", len(ret['train']))
def make_user_history(arr):
    ret = [ set() for _ in range(num_users) ]
    for u, i, r in arr:
        ret[int(u)].add(int(i))
    return ret

ret['train_positive_set'] = make_user_history(ret['train'])
ret['val_positive_set'] = make_user_history(ret['val'])
ret['test_positive_set'] = make_user_history(ret['test'])

ret['train_matrix'] = csr_matrix(
    ( np.ones(ret['train'].shape[0]), (ret['train'][:, 0].astype(np.int32), ret['train'][:, 1].astype(np.int32)) ),
    shape = (num_users, num_items)
)

ret['val_matrix'] = csr_matrix(
    ( np.ones(ret['val'].shape[0]), (ret['val'][:, 0].astype(np.int32), ret['val'][:, 1].astype(np.int32)) ),
    shape = (num_users, num_items)
)
ret['test_matrix'] = csr_matrix(
    ( np.ones(ret['test'].shape[0]), (ret['test'][:, 0].astype(np.int32), ret['test'][:, 1].astype(np.int32)) ),
    shape = (num_users, num_items)
)
ret.update({
    'num_users': num_users,
    'num_items': num_items,
    'num_interactions': len(ret['train']),
})


[1 1 0 ... 0 0 2] [[   0 1193    1]
 [   0  661    1]
 [   0  914    1]
 ...
 [6039  562    1]
 [6039 1096    1]
 [6039 1097    1]]
[1 1 0 ... 0 0 2] [[   0 1104    1]
 [   0  639    1]
 [   0  853    1]
 ...
 [6039  548    1]
 [6039 1024    1]
 [6039 1025    1]]
# users: 6040
# items: 3706
# interactions: 791718


In [ ]:
ret['negatives'] = [ set() for _ in range(num_users) ]
for u in range(num_users):
    while len(ret['negatives'][u]) < 50:
        rand_item = np.random.randint(0, num_items)
        if rand_item in ret['train_positive_set'][u]: continue
        if rand_item in ret['test_positive_set'][u]: continue
        ret['negatives'][u].add(rand_item)
    ret['negatives'][u] = list(ret['negatives'][u])
ret['negatives'] = np.array(ret['negatives'], dtype=np.int32)


In [ ]:
URM_train=ret['train_matrix']
URM_test=ret['test_matrix']
URM_validation=ret['val_matrix']

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
def average_precision(is_relevant, pos_items):

    if len(is_relevant) == 0:
        a_p = 0.0
    else:
        p_at_k = is_relevant * np.cumsum(is_relevant, dtype=np.float32) / (1 + np.arange(is_relevant.shape[0]))
        a_p = np.sum(p_at_k) / np.min([pos_items.shape[0], is_relevant.shape[0]])

    assert 0 <= a_p <= 1, a_p
    return a_p
class Metrics_Object(object):
    """
    Abstract class that should be used as superclass of all metrics requiring an object, therefore a state, to be computed
    """
    def __init__(self):
        pass

    def add_recommendations(self, recommended_items_ids):
        raise NotImplementedError()

    def get_metric_value(self):
        raise NotImplementedError()

    def merge_with_other(self, other_metric_object):
        raise NotImplementedError()
class MAP(Metrics_Object):
    """
    Mean Average Precision, defined as the mean of the AveragePrecision over all users
    """

    def __init__(self):
        super(MAP, self).__init__()
        self.cumulative_AP = 0.0
        self.n_users = 0

    def add_recommendations(self, is_relevant, pos_items):
        self.cumulative_AP += average_precision(is_relevant, pos_items)
        self.n_users += 1

    def get_metric_value(self):
        return self.cumulative_AP/self.n_users

    def merge_with_other(self, other_metric_object):
        assert other_metric_object is MAP, "MAP: attempting to merge with a metric object of different type"

        self.cumulative_AP += other_metric_object.cumulative_AP
        self.n_users += other_metric_object.n_users
def precision(is_relevant):

    if len(is_relevant) == 0:
        precision_score = 0.0
    else:
        precision_score = np.sum(is_relevant, dtype=np.float32) / len(is_relevant)

    assert 0 <= precision_score <= 1, precision_score
    return precision_score
def rmse(all_items_predicted_ratings, relevant_items, relevant_items_rating):

    relevant_items_error = (all_items_predicted_ratings[relevant_items]-relevant_items_rating)**2

    finite_prediction_mask = np.isfinite(relevant_items_error)

    if finite_prediction_mask.sum() == 0:
        rmse = np.nan

    else:
        relevant_items_error = relevant_items_error[finite_prediction_mask]

        squared_error = np.sum(relevant_items_error)


        mean_squared_error = squared_error/finite_prediction_mask.sum()
        rmse = np.sqrt(mean_squared_error)

    return rmse


def recall(is_relevant, pos_items):

    recall_score = np.sum(is_relevant, dtype=np.float32) / pos_items.shape[0]

    assert 0 <= recall_score <= 1, recall_score
    return recall_score
def ndcg(ranked_list, pos_items, relevance=None, at=None):

    if relevance is None:
        relevance = np.ones_like(pos_items)
    assert len(relevance) == pos_items.shape[0]

    it2rel = {it: r for it, r in zip(pos_items, relevance)}

    rank_scores = np.asarray([it2rel.get(it, 0.0) for it in ranked_list[:at]], dtype=np.float32)

    ideal_dcg = dcg(np.sort(relevance)[::-1])

    rank_dcg = dcg(rank_scores)

    if rank_dcg == 0.0:
        return 0.0

    ndcg_ = rank_dcg / ideal_dcg
    return ndcg_


def dcg(scores):
    return np.sum(np.divide(np.power(2, scores) - 1, np.log(np.arange(scores.shape[0], dtype=np.float32) + 2)),
                  dtype=np.float32)



In [ ]:
import numpy as np
import scipy.sparse as sps
import time, sys, copy

from enum import Enum
class EvaluatorMetrics(Enum):

    PRECISION = "PRECISION"
    RECALL = "RECALL"
    MAP = "MAP"
    NDCG = "NDCG"
    F1 = "F1"
    HIT_RATE = "HIT_RATE"
    RMSE = "RMSE"



def create_empty_metrics_dict(n_items, n_users, URM_train, ignore_items, ignore_users, cutoff, diversity_similarity_object):

    empty_dict = {}

    for metric in EvaluatorMetrics:

        if metric == EvaluatorMetrics.MAP:
            empty_dict[metric.value] = MAP()

        else:
            empty_dict[metric.value] = 0.0

    return  empty_dict

def get_result_string(results_run, n_decimals=7):

    output_str = ""

    for cutoff in results_run.keys():

        results_run_current_cutoff = results_run[cutoff]

        output_str += "CUTOFF: {} - ".format(cutoff)

        for metric in results_run_current_cutoff.keys():
            output_str += "{}: {:.{n_decimals}f}, ".format(metric, results_run_current_cutoff[metric], n_decimals = n_decimals)

        output_str += "\n"

    return output_str



class Evaluator(object):
    """Abstract Evaluator"""

    EVALUATOR_NAME = "Evaluator_Base_Class"

    def __init__(self, URM_test_list, cutoff_list, minRatingsPerUser=1, exclude_seen=np.bool_(True),
                        diversity_object = None,
                        ignore_items = None,
                        ignore_users = None):

        super(Evaluator, self).__init__()



        if ignore_items is None:
            self.ignore_items_flag = False
            self.ignore_items_ID = np.array([])
        else:
            print("Ignoring {} Items".format(len(ignore_items)))
            self.ignore_items_flag = True
            self.ignore_items_ID = np.array(ignore_items)

        self.cutoff_list = cutoff_list.copy()
        self.max_cutoff = max(self.cutoff_list)

        self.minRatingsPerUser = minRatingsPerUser
        self.exclude_seen = exclude_seen

        if not isinstance(URM_test_list, list):
            self.URM_test = URM_test_list.copy()
            URM_test_list = [URM_test_list]
        else:
            raise ValueError("List of URM_test not supported")

        self.diversity_object = diversity_object

        self.n_users, self.n_items = URM_test_list[0].shape

        self.URM_test_list = []
        usersToEvaluate_mask = np.zeros(self.n_users, dtype=np.bool)

        for URM_test in URM_test_list:

            URM_test = sps.csr_matrix(URM_test)
            self.URM_test_list.append(URM_test)

            rows = URM_test.indptr
            numRatings = np.ediff1d(rows)
            new_mask = numRatings >= minRatingsPerUser

            usersToEvaluate_mask = np.logical_or(usersToEvaluate_mask, new_mask)

        self.usersToEvaluate = np.arange(self.n_users)[usersToEvaluate_mask]


        if ignore_users is not None:
            print("Ignoring {} Users".format(len(ignore_users)))
            self.ignore_users_ID = np.array(ignore_users)
            self.usersToEvaluate = set(self.usersToEvaluate) - set(ignore_users)
        else:
            self.ignore_users_ID = np.array([])


        self.usersToEvaluate = list(self.usersToEvaluate)

    def get_user_relevant_items(self, user_id):

        assert self.URM_test.getformat() == "csr", "Evaluator_Base_Class: URM_test is not CSR, this will cause errors in getting relevant items"

        return self.URM_test.indices[self.URM_test.indptr[user_id]:self.URM_test.indptr[user_id+1]]


    def get_user_test_ratings(self, user_id):

        assert self.URM_test.getformat() == "csr", "Evaluator_Base_Class: URM_test is not CSR, this will cause errors in relevant items ratings"

        return self.URM_test.data[self.URM_test.indptr[user_id]:self.URM_test.indptr[user_id+1]]



class EvaluatorHoldout(Evaluator):
    """EvaluatorHoldout"""

    EVALUATOR_NAME = "EvaluatorHoldout"

    def __init__(self, URM_test_list, cutoff_list, minRatingsPerUser=1, exclude_seen=np.bool_(True),
                 diversity_object = None,
                 ignore_items = None,
                 ignore_users = None):


        super(EvaluatorHoldout, self).__init__(URM_test_list, cutoff_list,
                                               diversity_object = diversity_object,
                                               minRatingsPerUser=minRatingsPerUser, exclude_seen=exclude_seen,
                                               ignore_items = ignore_items, ignore_users = ignore_users)





    def _run_evaluation_on_selected_users(self, recommender_object, usersToEvaluate, block_size = None):


        if block_size is None:
            block_size = min(1000, int(1e8/self.n_items))



        start_time = time.time()
        start_time_print = time.time()


        results_dict = {}

        for cutoff in self.cutoff_list:
            results_dict[cutoff] = create_empty_metrics_dict(self.n_items, self.n_users,
                                                             URM_train,
                                                             self.ignore_items_ID,
                                                             self.ignore_users_ID,
                                                             cutoff,
                                                             self.diversity_object)

        n_users_evaluated = 0

        # Start from -block_size to ensure it to be 0 at the first block
        user_batch_start = 0
        user_batch_end = 0

        while user_batch_start < len(self.usersToEvaluate):

            user_batch_end = user_batch_start + block_size
            user_batch_end = min(user_batch_end, len(usersToEvaluate))

            test_user_batch_array = np.array(usersToEvaluate[user_batch_start:user_batch_end])
            user_batch_start = user_batch_end

            recommended_items_batch_list, scores_batch = recommender_object.recommend(test_user_batch_array,
                                                                      remove_seen_flag=self.exclude_seen,
                                                                      cutoff = self.max_cutoff,
                                                                      remove_top_pop_flag=False,
                                                                      remove_CustomItems_flag=self.ignore_items_flag,
                                                                      return_scores = True
                                                                     )


            assert len(recommended_items_batch_list) == len(test_user_batch_array), "{}: recommended_items_batch_list contained recommendations for {} users, expected was {}".format(
                self.EVALUATOR_NAME, len(recommended_items_batch_list), len(test_user_batch_array))

            assert scores_batch.shape[0] == len(test_user_batch_array), "{}: scores_batch contained scores for {} users, expected was {}".format(
                self.EVALUATOR_NAME, scores_batch.shape[0], len(test_user_batch_array))

            assert scores_batch.shape[1] == self.n_items, "{}: scores_batch contained scores for {} items, expected was {}".format(
                self.EVALUATOR_NAME, scores_batch.shape[1], self.n_items)


            for batch_user_index in range(len(recommended_items_batch_list)):

                test_user = test_user_batch_array[batch_user_index]

                relevant_items = self.get_user_relevant_items(test_user)
                relevant_items_rating = self.get_user_test_ratings(test_user)

                all_items_predicted_ratings = scores_batch[batch_user_index]
                user_rmse = rmse(all_items_predicted_ratings, relevant_items, relevant_items_rating)

                # Being the URM CSR, the indices are the non-zero column indexes
                recommended_items = recommended_items_batch_list[batch_user_index]
                is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)

                n_users_evaluated += 1

                for cutoff in self.cutoff_list:

                    results_current_cutoff = results_dict[cutoff]

                    is_relevant_current_cutoff = is_relevant[0:cutoff]
                    recommended_items_current_cutoff = recommended_items[0:cutoff]

                    results_current_cutoff[EvaluatorMetrics.PRECISION.value]            += precision(is_relevant_current_cutoff)
                    results_current_cutoff[EvaluatorMetrics.RECALL.value]               += recall(is_relevant_current_cutoff, relevant_items)
                    results_current_cutoff[EvaluatorMetrics.NDCG.value]                 += ndcg(recommended_items_current_cutoff, relevant_items, relevance=self.get_user_test_ratings(test_user), at=cutoff)
                    results_current_cutoff[EvaluatorMetrics.HIT_RATE.value]             += is_relevant_current_cutoff.sum()
                    results_current_cutoff[EvaluatorMetrics.RMSE.value]                 += user_rmse
                    results_current_cutoff[EvaluatorMetrics.MAP.value].add_recommendations(is_relevant_current_cutoff, relevant_items)
                    

                    sys.stdout.flush()
                    sys.stderr.flush()

                    start_time_print = time.time()



        return results_dict, n_users_evaluated




    def evaluateRecommender(self, recommender_object):

        if self.ignore_items_flag:
            recommender_object.set_items_to_ignore(self.ignore_items_ID)



        results_dict, n_users_evaluated = self._run_evaluation_on_selected_users(recommender_object, self.usersToEvaluate)


        if (n_users_evaluated > 0):

            for cutoff in self.cutoff_list:

                results_current_cutoff = results_dict[cutoff]

                for key in results_current_cutoff.keys():

                    value = results_current_cutoff[key]

                    if isinstance(value, Metrics_Object):
                        results_current_cutoff[key] = value.get_metric_value()
                    else:
                        results_current_cutoff[key] = value/n_users_evaluated

                precision_ = results_current_cutoff[EvaluatorMetrics.PRECISION.value]
                recall_ = results_current_cutoff[EvaluatorMetrics.RECALL.value]

                if precision_ + recall_ != 0:
                    results_current_cutoff[EvaluatorMetrics.F1.value] = 2 * (precision_ * recall_) / (precision_ + recall_)


        else:
            print("WARNING: No users had a sufficient number of relevant items")



        results_run_string = get_result_string(results_dict)




        if self.ignore_items_flag:
            recommender_object.reset_items_to_ignore()


        return (results_dict, results_run_string)




In [ ]:
evaluatorValidation = EvaluatorHoldout(URM_validation,[5,10,50,80,100,500],exclude_seen=True)

<ipython-input-11-6ceb33196298>:90: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  usersToEvaluate_mask = np.zeros(self.n_users, dtype=np.bool)


In [ ]:
class EarlyStoppingScheduler(object):
    def __init__(self, model, evaluator, metrics=['PRECISION', 'RECALL', 'MAP', 'NDCG'], freq=1, allow_worse=5):
        self.model = model
        self.evaluator = evaluator
        self.metrics = metrics
        self.freq = freq
        self.best_scores = np.zeros(len(metrics))
        self.allow_worse = allow_worse
        self.worse_left = allow_worse
        self.scores = []

    def score(self, epoch):
        if epoch % self.freq == 0:
            results_dic, _ = self.evaluator.evaluateRecommender(self.model) #TODO: dependent on recommender interface
            curr_scores = np.array([results_dic[5][m] for m in self.metrics])
            self.scores.append(curr_scores)
            if np.all(np.less_equal(curr_scores, self.best_scores)):
                if self.worse_left > 0:
                    self.worse_left -= 1
                else:
                    self.model.stop_fit()
                    self.model.load_model()
            else:
                self.best_scores = curr_scores
                self.worse_left = self.allow_worse
                self.model.save_current_model()

    def __call__(self, epoch):
        self.score(epoch)

    def load_best(self):
        self.model.load_model()

    def get_scores(self):
        return self.scores

In [ ]:

import numpy as np
import os


class BaseRecommender(object):

    RECOMMENDER_NAME = "Recommender_Base_Class"

    def __init__(self, URM_train):

        super(BaseRecommender, self).__init__()

        self.URM_train = URM_train.copy()
        self.URM_train.eliminate_zeros()

        self.n_users, self.n_items = self.URM_train.shape

        self.normalize = False

        self.filterTopPop = False
        self.filterTopPop_ItemsID = np.array([], dtype=np.int)

        self.items_to_ignore_flag = False
        self.items_to_ignore_ID = np.array([], dtype=np.int)

        self._cold_user_mask = np.ediff1d(self.URM_train.indptr) == 0

        if self._cold_user_mask.any():
            print("{}: Detected {} ({:.2f} %) cold users.".format(
                self.RECOMMENDER_NAME, self._cold_user_mask.sum(), self._cold_user_mask.sum()/len(self._cold_user_mask)*100))



    def _get_cold_user_mask(self):
        return self._cold_user_mask

    def get_URM_train(self):
        return self.URM_train.copy()

    def set_URM_train(self, URM_train_new, **kwargs):

        assert self.URM_train.shape == URM_train_new.shape, "{}: set_URM_train old and new URM train have different shapes".format(self.RECOMMENDER_NAME)

        if len(kwargs)>0:
            print("{}: set_URM_train keyword arguments not supported for this recommender class. Received: {}".format(self.RECOMMENDER_NAME, kwargs))

        self.URM_train = URM_train_new.copy()
        self.URM_train.eliminate_zeros()

        self._cold_user_mask = np.ediff1d(self.URM_train.indptr) == 0

        if self._cold_user_mask.any():
            print("{}: Detected {} ({:.2f} %) cold users.".format(
                self.RECOMMENDER_NAME, self._cold_user_mask.sum(), self._cold_user_mask.sum()/len(self._cold_user_mask)*100))



    def set_items_to_ignore(self, items_to_ignore):

        self.items_to_ignore_flag = True
        self.items_to_ignore_ID = np.array(items_to_ignore, dtype=np.int)

    def reset_items_to_ignore(self):

        self.items_to_ignore_flag = False
        self.items_to_ignore_ID = np.array([], dtype=np.int)


    def _remove_TopPop_on_scores(self, scores_batch):
        scores_batch[:, self.filterTopPop_ItemsID] = -np.inf
        return scores_batch


    def _remove_CustomItems_on_scores(self, scores_batch):
        scores_batch[:, self.items_to_ignore_ID] = -np.inf
        return scores_batch


    def _remove_seen_on_scores(self, user_id, scores):

        assert self.URM_train.getformat() == "csr", "Recommender_Base_Class: URM_train is not CSR, this will cause errors in filtering seen items"

        if user_id >= len(self.URM_train.indptr):
          seen = []
        elif user_id + 1 >= len(self.URM_train.indptr):
          seen = self.URM_train.indices[self.URM_train.indptr[user_id]:]
        else:
          seen = self.URM_train.indices[self.URM_train.indptr[user_id]:self.URM_train.indptr[user_id + 1]]
        scores[seen] = -np.inf
        return scores



    def recommend(self, user_id_array, cutoff = None, remove_seen_flag=True, items_to_compute = None,
                  remove_top_pop_flag = False, remove_CustomItems_flag = False, return_scores = False):

        if np.isscalar(user_id_array):
            user_id_array = np.atleast_1d(user_id_array)
            single_user = True
        else:
            single_user = False


        if cutoff is None:
            cutoff = self.URM_train.shape[1] - 1

        scores_batch = self._compute_item_score(user_id_array, items_to_compute=items_to_compute)

        for user_index in range(len(user_id_array)):

            user_id = user_id_array[user_index]

            if remove_seen_flag:
                scores_batch[user_index,:] = self._remove_seen_on_scores(user_id, scores_batch[user_index, :])


        if remove_top_pop_flag:
            scores_batch = self._remove_TopPop_on_scores(scores_batch)

        if remove_CustomItems_flag:
            scores_batch = self._remove_CustomItems_on_scores(scores_batch)
        relevant_items_partition = (-scores_batch).argpartition(cutoff, axis=1)[:,0:cutoff]
        relevant_items_partition_original_value = scores_batch[np.arange(scores_batch.shape[0])[:, None], relevant_items_partition]
        relevant_items_partition_sorting = np.argsort(-relevant_items_partition_original_value, axis=1)
        ranking = relevant_items_partition[np.arange(relevant_items_partition.shape[0])[:, None], relevant_items_partition_sorting]

        ranking_list = [None] * ranking.shape[0]

        for user_index in range(len(user_id_array)):
            user_recommendation_list = ranking[user_index]
            user_item_scores = scores_batch[user_index, user_recommendation_list]

            not_inf_scores_mask = np.logical_not(np.isinf(user_item_scores))

            user_recommendation_list = user_recommendation_list[not_inf_scores_mask]
            ranking_list[user_index] = user_recommendation_list.tolist()

        if single_user:
            ranking_list = ranking_list[0]


        if return_scores:
            return ranking_list, scores_batch

        else:
            return ranking_list


In [ ]:
class GANMF(BaseRecommender):

    def __init__(self, URM_train, mode='user', verbose=False, seed=1234, is_experiment=False):

        if mode not in ['user', 'item']:
            raise ValueError('Accepted training modes are `user` and `item`. Given was {}.', mode)

        self.mode = mode
        if self.mode == 'item':
            self.URM_train = URM_train.T.tocsr()
        else:
            self.URM_train = URM_train
        self.num_users, self.num_items = self.URM_train.shape
        self.config = None
        self.seed = seed
        self.verbose = verbose
        
    def build(self, num_factors=10, emb_dim=32):
        glorot_uniform = tf.glorot_uniform_initializer()

        def autoencoder(input_data):
            with tf.variable_scope('autoencoder', reuse=tf.AUTO_REUSE):
                encoding = tf.layers.dense(input_data, units=emb_dim, kernel_initializer=glorot_uniform, name='encoding')
                decoding = tf.layers.dense(encoding, units=self.num_items, kernel_initializer=glorot_uniform, name='decoding')
            loss = tf.losses.mean_squared_error(input_data, decoding)
            return encoding, loss

        def generator(condition):
            with tf.variable_scope('generator', reuse=tf.AUTO_REUSE):
                user_embeddings = tf.get_variable(shape=[self.num_users, num_factors], trainable=True,
                                initializer=glorot_uniform, name='user_embeddings')
                item_embeddings = tf.get_variable(shape=[self.num_items, num_factors], trainable=True,
                                initializer=glorot_uniform, name='item_embeddings')
            condition = tf.where(tf.less(condition, self.num_users-1), condition, tf.zeros_like(condition))
            user_lookup = tf.nn.embedding_lookup(user_embeddings, condition)
            item_factors = tf.layers.dense(user_lookup, units=num_factors, activation=None, kernel_initializer=glorot_uniform, name='item_factors')
            #fake_data = tf.matmul(item_factors[:, :, None], item_embeddings[None, :, :], transpose_b=True)[:, 0, :]

            fake_data = tf.matmul(tf.squeeze(user_lookup, axis=1), item_embeddings, transpose_b=True)
            return fake_data

        self.autoencoder, self.generator = autoencoder, generator

    def fit(self, num_factors=10, emb_dim=32, epochs=300, batch_size=32, d_lr=1e-4, g_lr=1e-4, d_steps=1, g_steps=1,
            d_reg=0, g_reg=0, m=1, recon_coefficient=1e-2, allow_worse=None, freq=None, metrics=['MAP','NDCG','RMSE'], sample_every=None,
            validation_evaluator=None, validation_set=None):

        # Construct the model config
        self.config = dict(locals())
        del self.config['self']

        # First clear the session to save GPU memory
        tf.reset_default_graph()
        # Set fixed seed for the TF graph
        tf.set_random_seed(self.seed)

        self.build(num_factors, emb_dim)

        # Create optimizers
        opt_gen = tf.train.AdamOptimizer(learning_rate=g_lr)
        opt_disc = tf.train.AdamOptimizer(learning_rate=d_lr)

        # placeholders
        real_profile = tf.placeholder(dtype=tf.float32, shape=[None, self.num_items])
        self.user_id = tf.placeholder(dtype=tf.int32, shape=[None, 1])

        # generator ops
        self.fake_profile = self.generator(self.user_id)

        # autoencoder ops
        real_encoding, real_recon_loss = self.autoencoder(real_profile)
        fake_encoding, fake_recon_loss = self.autoencoder(self.fake_profile)

        # model parameters
        self.dvars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='autoencoder')
        self.gvars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='generator')

        with tf.variable_scope('dvars_best', reuse=tf.AUTO_REUSE):
            self.dvars_best = []
            for idx, var in enumerate(self.dvars):
                self.dvars_best.append(tf.get_variable('dva_r' + str(idx), shape=var.get_shape(), trainable=False))
        with tf.variable_scope('gvars_best', reuse=tf.AUTO_REUSE):
            self.gvars_best = []
            for idx, var in enumerate(self.gvars):
                self.gvars_best.append(tf.get_variable('gvar_' + str(idx), shape=var.get_shape(), trainable=False))

        # losses
        dloss = real_recon_loss + tf.maximum(0.0, m * real_recon_loss - fake_recon_loss) + \
                d_reg * tf.add_n([tf.nn.l2_loss(var) for var in self.dvars])
        gloss = (1 - recon_coefficient) * fake_recon_loss + \
                recon_coefficient * tf.losses.mean_squared_error(real_encoding, fake_encoding) + \
                g_reg * tf.add_n([tf.nn.l2_loss(var) for var in self.gvars])

        # update ops
        dtrain = opt_disc.minimize(dloss, var_list=self.dvars)
        gtrain = opt_gen.minimize(gloss, var_list=self.gvars)

        self.sess = tf.Session()
        self.sess.run(tf.initialize_all_variables())

        self._stop_training = False
        if validation_evaluator is not None:
            early_stop = EarlyStoppingScheduler(self, evaluator=validation_evaluator, allow_worse=allow_worse,
                                                freq=freq, metrics=metrics)

        all_users = np.array(range(self.num_users))
        step = batch_size

        train_g_loss = []
        train_d_loss = []

        if self.verbose:
            print('Starting training...')

        t_start = time.time()
        e_start = time.time()

        epoch = 1

        pbar = tqdm.tqdm(total=epochs, initial=1)

        while not self._stop_training and epoch < epochs + 1:
            batch_d_loss = []
            batch_g_loss = []
            np.random.shuffle(all_users)
            for _ in range(d_steps):
                start_idx = 0
                while start_idx < len(all_users):
                    end_idx = start_idx + step
                    if end_idx > len(all_users):
                        end_idx = len(all_users)

                    uids = all_users[start_idx: end_idx]
                    real_histories = self.URM_train[uids].toarray()

                    _, _dloss = self.sess.run([dtrain, dloss], {real_profile: real_histories, self.user_id: uids.reshape(-1, 1)})
                    batch_d_loss.append(_dloss)
                    start_idx = end_idx

            for _ in range(g_steps):
                start_idx = 0
                while start_idx < len(all_users):
                    end_idx = start_idx + step
                    if end_idx > len(all_users):
                        end_idx = len(all_users)

                    uids = all_users[start_idx: end_idx]
                    real_histories = self.URM_train[uids].toarray()
                    _, _gloss = self.sess.run([gtrain, gloss], {real_profile: real_histories, self.user_id: uids.reshape(-1, 1)})
                    batch_g_loss.append(_gloss)
                    start_idx = end_idx

            mean_epoch_g_loss = np.mean(batch_g_loss)
            mean_epoch_d_loss = np.mean(batch_d_loss)

            train_g_loss.append(mean_epoch_g_loss)
            train_d_loss.append(mean_epoch_d_loss)

            if validation_set is not None and sample_every is not None and epoch % sample_every == 0:
                t_end = time.time()
                total = t_end-e_start
                print('Epoch : {:d}. Total: {:.2f} secs, {:.2f} secs/epoch.'.format(epoch, total, total/sample_every))
                if self.mode == 'item':
                    self.URM_train = self.URM_train.T.tocsr()
                _, results_run_string = validation_evaluator.evaluateRecommender(self)
                if self.mode == 'item':
                    self.URM_train = self.URM_train.T.tocsr()
                print(results_run_string)
                e_start = time.time()

            if validation_evaluator is not None:
                if self.mode == 'item':
                    self.URM_train = self.URM_train.T.tocsr()
                early_stop(epoch)
                if self.mode == 'item':
                    self.URM_train = self.URM_train.T.tocsr()

                if self._stop_training:
                    print('Training stopped, epoch:', epoch)

            epoch += 1
            pbar.update()
        pbar.close()

        t_end = time.time()
        if self.verbose:
            print('Training took {:.2f} seconds'.format(t_end - t_start))

        if self.mode == 'item':
            self.URM_train = self.URM_train.T.tocsr()

        return epoch

    def stop_fit(self):
        self._stop_training = True

    def save_current_model(self):
        for idx, var in enumerate(self.dvars):
            self.sess.run(self.dvars_best[idx].assign(var))
        for idx, var in enumerate(self.gvars):
            self.sess.run(self.gvars_best[idx].assign(var))

    def load_model(self):
        for idx, var in enumerate(self.dvars_best):
            self.sess.run(self.dvars[idx].assign(var))
        for idx, var in enumerate(self.gvars_best):
            self.sess.run(self.gvars[idx].assign(var))

    def _compute_item_score(self, user_id_array, items_to_compute=None):
        if self.mode == 'item':
            predictions = self.sess.run(self.fake_profile, {self.user_id: np.array(range(self.num_users)).reshape(-1, 1)})
            return predictions.transpose()[user_id_array]
        else:
            return self.sess.run(self.fake_profile, {self.user_id: user_id_array.reshape(-1, 1)})

In [ ]:
import tqdm

In [ ]:
gan = GANMF(URM_train, mode='user')
evaluatorValidation = EvaluatorHoldout(URM_validation,[5,10,20,50,80,100],exclude_seen=np.bool_(True))
gan.fit(num_factors=250,
        emb_dim=461,
        d_reg=0.0001,
        g_reg=0,
        epochs=300,
        batch_size=512,
        g_lr=0.0001,
        d_lr=0.0001,
        d_steps=1,
        g_steps=1,
        recon_coefficient=0.1728565040890659,
        m=10,
        allow_worse=5,
        freq=5,
        validation_evaluator=evaluatorValidation,
        sample_every=10,
        validation_set=URM_validation)

<ipython-input-11-6ceb33196298>:90: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  usersToEvaluate_mask = np.zeros(self.n_users, dtype=np.bool)
Instructions for updating:
Use `tf.global_variables_initializer` instead.
  3%|▎         | 10/300 [00:50<18:03,  3.74s/it]

Epoch : 10. Total: 51.21 secs, 5.12 secs/epoch.
CUTOFF: 5 - PRECISION: 0.0071854, RECALL: 0.0020573, MAP: 0.0033860, NDCG: 0.0027812, F1: 0.0031988, HIT_RATE: 0.0359272, RMSE: 0.9991374, 
CUTOFF: 10 - PRECISION: 0.0073510, RECALL: 0.0039277, MAP: 0.0024869, NDCG: 0.0042759, F1: 0.0051198, HIT_RATE: 0.0735099, RMSE: 0.9991374, 
CUTOFF: 20 - PRECISION: 0.0070199, RECALL: 0.0075796, MAP: 0.0019842, NDCG: 0.0064913, F1: 0.0072890, HIT_RATE: 0.1403974, RMSE: 0.9991374, 
CUTOFF: 50 - PRECISION: 0.0066159, RECALL: 0.0173308, MAP: 0.0018877, NDCG: 0.0112384, F1: 0.0095762, HIT_RATE: 0.3307947, RMSE: 0.9991374, 
CUTOFF: 80 - PRECISION: 0.0064031, RECALL: 0.0272266, MAP: 0.0020357, NDCG: 0.0152082, F1: 0.0103680, HIT_RATE: 0.5122517, RMSE: 0.9991374, 
CUTOFF: 100 - PRECISION: 0.0063626, RECALL: 0.0340909, MAP: 0.0021463, NDCG: 0.0177370, F1: 0.0107237, HIT_RATE: 0.6362583, RMSE: 0.9991374, 



  7%|▋         | 20/300 [02:59<20:34,  4.41s/it]

Epoch : 20. Total: 87.00 secs, 8.70 secs/epoch.
CUTOFF: 5 - PRECISION: 0.0315232, RECALL: 0.0079215, MAP: 0.0172365, NDCG: 0.0124541, F1: 0.0126613, HIT_RATE: 0.1576159, RMSE: 0.9865058, 
CUTOFF: 10 - PRECISION: 0.0293212, RECALL: 0.0146492, MAP: 0.0121911, NDCG: 0.0179367, F1: 0.0195373, HIT_RATE: 0.2932119, RMSE: 0.9865058, 
CUTOFF: 20 - PRECISION: 0.0270778, RECALL: 0.0278191, MAP: 0.0094649, NDCG: 0.0261808, F1: 0.0274435, HIT_RATE: 0.5415563, RMSE: 0.9865058, 
CUTOFF: 50 - PRECISION: 0.0229636, RECALL: 0.0602300, MAP: 0.0089193, NDCG: 0.0419242, F1: 0.0332501, HIT_RATE: 1.1481788, RMSE: 0.9865058, 
CUTOFF: 80 - PRECISION: 0.0204967, RECALL: 0.0868345, MAP: 0.0095877, NDCG: 0.0528017, F1: 0.0331650, HIT_RATE: 1.6397351, RMSE: 0.9865058, 
CUTOFF: 100 - PRECISION: 0.0196589, RECALL: 0.1044701, MAP: 0.0101109, NDCG: 0.0594884, F1: 0.0330909, HIT_RATE: 1.9658940, RMSE: 0.9865058, 



 10%|█         | 30/300 [05:08<20:03,  4.46s/it]

...

Epoch : 290. Total: 86.25 secs, 8.62 secs/epoch.
CUTOFF: 5 - PRECISION: 0.2031126, RECALL: 0.0978055, MAP: 0.1376375, NDCG: 0.1216951, F1: 0.1320328, HIT_RATE: 1.0155629, RMSE: 0.7936658, 
CUTOFF: 10 - PRECISION: 0.1779967, RECALL: 0.1625929, MAP: 0.1168769, NDCG: 0.1652689, F1: 0.1699465, HIT_RATE: 1.7799669, RMSE: 0.7936658, 
CUTOFF: 20 - PRECISION: 0.1495116, RECALL: 0.2532338, MAP: 0.1119034, NDCG: 0.2152407, F1: 0.1880165, HIT_RATE: 2.9902318, RMSE: 0.7936658, 
CUTOFF: 50 - PRECISION: 0.1083874, RECALL: 0.4128285, MAP: 0.1210575, NDCG: 0.2877304, F1: 0.1716963, HIT_RATE: 5.4193709, RMSE: 0.7936658, 
CUTOFF: 80 - PRECISION: 0.0877049, RECALL: 0.5032417, MAP: 0.1293576, NDCG: 0.3242348, F1: 0.1493765, HIT_RATE: 7.0163907, RMSE: 0.7936658, 
CUTOFF: 100 - PRECISION: 0.0783940, RECALL: 0.5475052, MAP: 0.1333506, NDCG: 0.3409824, F1: 0.1371503, HIT_RATE: 7.8394040, RMSE: 0.7936658, 



100%|██████████| 300/300 [1:03:06<00:00,  4.43s/it]

Epoch : 300. Total: 86.47 secs, 8.65 secs/epoch.
CUTOFF: 5 - PRECISION: 0.2004305, RECALL: 0.0963719, MAP: 0.1360153, NDCG: 0.1201990, F1: 0.1301598, HIT_RATE: 1.0021523, RMSE: 0.7958023, 
CUTOFF: 10 - PRECISION: 0.1765894, RECALL: 0.1611522, MAP: 0.1157812, NDCG: 0.1638546, F1: 0.1685180, HIT_RATE: 1.7658940, RMSE: 0.7958023, 
CUTOFF: 20 - PRECISION: 0.1485430, RECALL: 0.2514688, MAP: 0.1108218, NDCG: 0.2135729, F1: 0.1867642, HIT_RATE: 2.9708609, RMSE: 0.7958023, 
CUTOFF: 50 - PRECISION: 0.1079570, RECALL: 0.4098707, MAP: 0.1199084, NDCG: 0.2856788, F1: 0.1709001, HIT_RATE: 5.3978477, RMSE: 0.7958023, 
CUTOFF: 80 - PRECISION: 0.0872517, RECALL: 0.4989942, MAP: 0.1280419, NDCG: 0.3216950, F1: 0.1485318, HIT_RATE: 6.9801325, RMSE: 0.7958023, 
CUTOFF: 100 - PRECISION: 0.0780298, RECALL: 0.5427726, MAP: 0.1320127, NDCG: 0.3383155, F1: 0.1364442, HIT_RATE: 7.8029801, RMSE: 0.7958023, 



301it [1:04:30, 12.90s/it]


301

In [ ]:
evaluator = EvaluatorHoldout(URM_test, [5,10,20,50,80,100], exclude_seen=True)
results_dic, results_run_string = evaluator.evaluateRecommender(gan)
print(results_run_string)

<ipython-input-11-6ceb33196298>:90: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  usersToEvaluate_mask = np.zeros(self.n_users, dtype=np.bool)


CUTOFF: 5 - PRECISION: 0.2365563, RECALL: 0.0996637, MAP: 0.1557893, NDCG: 0.1335228, F1: 0.1402420, HIT_RATE: 1.1827815, RMSE: 0.7959542, 
CUTOFF: 10 - PRECISION: 0.2035430, RECALL: 0.1629253, MAP: 0.1278672, NDCG: 0.1791633, F1: 0.1809832, HIT_RATE: 2.0354305, RMSE: 0.7959542, 
CUTOFF: 20 - PRECISION: 0.1691639, RECALL: 0.2550325, MAP: 0.1208024, NDCG: 0.2327748, F1: 0.2034072, HIT_RATE: 3.3832781, RMSE: 0.7959542, 
CUTOFF: 50 - PRECISION: 0.1210132, RECALL: 0.4128077, MAP: 0.1306930, NDCG: 0.3088916, F1: 0.1871609, HIT_RATE: 6.0506623, RMSE: 0.7959542, 
CUTOFF: 80 - PRECISION: 0.0969578, RECALL: 0.5016631, MAP: 0.1393060, NDCG: 0.3465405, F1: 0.1625073, HIT_RATE: 7.7566225, RMSE: 0.7959542, 
CUTOFF: 100 - PRECISION: 0.0862351, RECALL: 0.5438527, MAP: 0.1435142, NDCG: 0.3634732, F1: 0.1488656, HIT_RATE: 8.6235099, RMSE: 0.7959542, 



In [ ]:
recommended_items_batch_list, scores_batch = gan.recommend(user_id_array=np.array([i for i in range(1,501)]),
                                                                      remove_seen_flag=True,
                                                                      cutoff = 500,
                                                                      remove_top_pop_flag=False,
                                                                      remove_CustomItems_flag=False,
                                                                      return_scores = True
                                                                     )


In [ ]:
import pandas as pd 
pd.DataFrame(recommended_items_batch_list).to_csv("generated.csv")

In [ ]:
recommended_items_batch_list=np.array(recommended_items_batch_list)

In [ ]:
import numpy as np

# assume preds is of shape (3000, 500)
recommended_items = []
for i in range(6040):
  a=[]
  for j in range(3706):
    try:
      a.append(recommended_items_batch_list[i,j])
    except:
      a.append(0)
  recommended_items.append(a)

In [ ]:
np.array(recommended_items).shape

(6040, 3706)

In [ ]:
import numpy as np

num_users = len(recommended_items)
num_items = np.max(np.array(recommended_items))+1

interaction_matrix = np.zeros((num_users, num_items))

for user_id, recommended_items_list in enumerate(recommended_items):
    interaction_matrix[user_id, recommended_items_list] = 1


In [ ]:
import numpy as np
from scipy.sparse import csr_matrix

# Assuming recommended_items_batch_list is a 2D numpy array
sparse_matrix = csr_matrix(interaction_matrix)


In [ ]:
sparse_matrix.shape

(6040, 3706)

In [ ]:
import jax
import functools
from jax import scipy as sp
from jax import numpy as jnp
from neural_tangents import stax

a,b,kernel_fn=stax.serial(stax.Flatten(),
    stax.Dense(1024,W_std=2*0.5,b_std=0.1,parameterization='ntk'),
    stax.Relu(),
    stax.Dense(num_items, W_std=2*0.5, b_std=0.1, parameterization='ntk'))
kernel_fn=functools.partial(kernel_fn,get='ntk')
@jax.jit
def kernelized_rr(X_train, X_predict, reg):
    K_train = kernel_fn(X_train, X_train)
    K_predict = kernel_fn(X_predict, X_train)
    K_reg = (K_train + jnp.abs(reg) * jnp.trace(K_train) * jnp.eye(K_train.shape[0]) / K_train.shape[0])     
    return jnp.dot(K_predict, sp.linalg.solve(K_reg, X_train, assume_a='pos'))

In [ ]:
train_x=sparse_matrix
sample_user=jnp.array(sparse_matrix[sparse_matrix.getnnz(1)>0].todense())

In [ ]:
preds1=kernelized_rr(sample_user,(sparse_matrix).todense(),reg=5.0)

In [ ]:
preds1.shape

(6040, 3706)

In [ ]:
logits=np.array(preds1)
train_positive_list = list(map(list, ret['train_positive_set']))
for b in range(len(logits)):logits[b][train_positive_list[b]]=-float(1e6)
indices = (-logits).argsort()[:, :100].tolist()
hr=0
for i in range(100):
  for b in range(len(logits)):
    num_pos = float(len((ret['val_positive_set']+ret['test_positive_set'])[b]))

    hr += float(len(set(indices[b][:100]) &(ret['val_positive_set']+ret['test_positive_set'])[b])) / float(min(num_pos, 100))
hr/=num_users
print("hit rate for top @100:",hr)
dcg,idcg=0.0,0.0
for i in range(len(logits)):
  for at, pred in enumerate(indices[b][:100]):
      if pred in (ret['test_positive_set'])[b]: 
          dcg += 1.0 / np.log2(at + 2)
      if at < num_pos: 
          idcg += 1.0 / np.log2(at + 2)
print('ndcg @100:',dcg/idcg)

hit rate for top @100: 17.510289820905207
ndcg @100: 0.15592148287128016


In [ ]:
from collections import defaultdict
def get_item_propensity(A = 0.55, B = 1.5):
    item_count = defaultdict(int)
    for u, i, r in sparse_matrix: item_count[i] += 1
    item_freq_map = item_count
    item_freq = [ item_freq_map[i] for i in range(ret['num_items']) ]
    num_instances = ret['num_interactions']

    C = (np.log(num_instances)-1)*np.power(B+1, A)
    wts = 1.0 + C*np.power(np.array(item_freq)+B, -A)
    return np.ravel(wts)

In [ ]:
!pip install neural_tangents

In [ ]:
ret['train_matrix']

<6040x3706 sparse matrix of type '<class 'numpy.float64'>'
	with 791718 stored elements in Compressed Sparse Row format>

In [ ]:
train_x=ret['train_matrix']
sample_user=jnp.array(ret['train_matrix'][ret['train_matrix'].getnnz(1)>0].todense())

In [ ]:
preds=kernelized_rr(sample_user,(ret['train_matrix']).todense(),reg=5.0)

In [ ]:
preds.shape

(6040, 3706)

In [ ]:
logits=np.array(preds)
train_positive_list = list(map(list, ret['train_positive_set']))
for b in range(len(logits)):logits[b][train_positive_list[b]]=-float(1e6)
indices = (-logits).argsort()[:, :100].tolist()
hr=0
for i in range(100):
  for b in range(len(logits)):
    num_pos = float(len((ret['val_positive_set']+ret['test_positive_set'])[b]))

    hr += float(len(set(indices[b][:100]) &(ret['val_positive_set']+ret['test_positive_set'])[b])) / float(min(num_pos, 100))
hr/=num_users
print("hit rate for top @100:",hr)

In [ ]:
dcg,idcg=0.0,0.0
for i in range(len(logits)):
  for at, pred in enumerate(indices[b][:100]):
      if pred in (ret['test_positive_set'])[b]: 
          dcg += 1.0 / np.log2(at + 2)
      if at < num_pos: 
          idcg += 1.0 / np.log2(at + 2)

In [ ]:
print('ndcg @100:',dcg/idcg)

ndcg @100: 0.4433405800945249


In [ ]:
import jax
import numpy as np
import jax.numpy as jnp
from numba import jit, float64

INF = float(1e6)

def evaluate( kernelized_rr_forward, data, item_propensity, train_x, topk = [ 10, 100 ], test_set_eval = False):
    preds, y_binary, metrics = [], [], {}
    for kind in [ 'HR', 'NDCG', 'PSP' ]:
        for k in topk: 
            metrics['{}@{}'.format(kind, k)] = 0.0

    train_positive_list = list(map(list, ret['train_positive_set']))
    if test_set_eval:
        for u in range(len(train_positive_list)): train_positive_list[u] += list(ret['val_positive_set'][u])

    eval_context = ret['train_matrix']
    if test_set_eval: eval_context += ret['val_matrix']

    to_predict = ret['val_positive_set']
    if test_set_eval: to_predict = ret['test_positive_set']

    bsz = 2000 
    for i in range(0, ret['num_users'], bsz):
        temp_preds = kernelized_rr_forward(train_x, eval_context[i:i+bsz].todense(), reg = 5.0)
        
        metrics, temp_preds, temp_y = evaluate_batch(
            ret['negatives'][i:i+bsz], np.array(temp_preds), 
            train_positive_list[i:i+bsz], to_predict[i:i+bsz], item_propensity, 
            topk, metrics
        )
        
        preds += temp_preds
        y_binary += temp_y

    y_binary, preds = np.array(y_binary), np.array(preds)
    if (True not in np.isnan(y_binary)) and (True not in np.isnan(preds)):
        metrics['AUC'] = round(fast_auc(y_binary, preds), 4)

    for kind in [ 'HR', 'NDCG', 'PSP' ]:
        for k in topk: 
            metrics['{}@{}'.format(kind, k)] = round(
                float(100.0 * metrics['{}@{}'.format(kind, k)]) / ret['num_users'], 4
            )

    metrics['num_users'] = int(train_x.shape[0])
    metrics['num_interactions'] = int(jnp.count_nonzero(train_x.astype(np.int8)))

    return metrics

def evaluate_batch(auc_negatives, logits, train_positive, test_positive_set, item_propensity, topk, metrics, train_metrics = False):
    # AUC Stuff
    temp_preds, temp_y = [], []
    for b in range(len(logits)):
        temp_preds += np.take(logits[b], np.array(list(test_positive_set[b]))).tolist()
        temp_y += [ 1.0 for _ in range(len(test_positive_set[b])) ]

        temp_preds += np.take(logits[b], auc_negatives[b]).tolist()
        temp_y += [ 0.0 for _ in range(len(auc_negatives[b])) ]

    # Marking train-set consumed items as negative INF
    for b in range(len(logits)): logits[b][ train_positive[b] ] = -INF

    indices = (-logits).argsort()[:, :max(topk)].tolist()

    for k in topk: 
        for b in range(len(logits)):
            num_pos = float(len(test_positive_set[b]))

            metrics['HR@{}'.format(k)] += float(len(set(indices[b][:k]) & test_positive_set[b])) / float(min(num_pos, k))

            test_positive_sorted_psp = sorted([ item_propensity[x] for x in test_positive_set[b] ])[::-1]

            dcg, idcg, psp, max_psp = 0.0, 0.0, 0.0, 0.0
            for at, pred in enumerate(indices[b][:k]):
                if pred in test_positive_set[b]: 
                    dcg += 1.0 / np.log2(at + 2)
                    psp += float(item_propensity[pred]) / float(min(num_pos, k))
                if at < num_pos: 
                    idcg += 1.0 / np.log2(at + 2)
                    max_psp += test_positive_sorted_psp[at]

            metrics['NDCG@{}'.format(k)] += dcg / idcg
            metrics['PSP@{}'.format(k)] += psp / max_psp

    return metrics, temp_preds, temp_y

@jit(float64(float64[:], float64[:]))
def fast_auc(y_true, y_prob):
    y_true = y_true[np.argsort(y_prob)]
    nfalse, auc = 0, 0
    for i in range(len(y_true)):
        nfalse += (1 - y_true[i])
        auc += y_true[i] * nfalse
    return auc / (nfalse * (len(y_true) - nfalse))

In [ ]:
from collections import defaultdict
def get_item_propensity(A = 0.55, B = 1.5):
    item_count = defaultdict(int)
    for u, i, r in ret['train']: item_count[i] += 1
    item_freq_map = item_count
    item_freq = [ item_freq_map[i] for i in range(ret['num_items']) ]
    num_instances = ret['num_interactions']

    C = (np.log(num_instances)-1)*np.power(B+1, A)
    wts = 1.0 + C*np.power(np.array(item_freq)+B, -A)
    return np.ravel(wts)

In [ ]:
topk=[10,100]

In [ ]:
train_positive_list = list(map(list, ret['train_positive_set']))

In [ ]:
item_propensity = get_item_propensity()

In [ ]:
test_metrics = evaluate( kernelized_rr, data, item_propensity, sample_user, test_set_eval = True)

In [ ]:
test_metrics

{'HR@10': 32.4454,
 'HR@100': 60.525,
 'NDCG@10': 33.6757,
 'NDCG@100': 43.0133,
 'PSP@10': 3.3146,
 'PSP@100': 6.6373,
 'AUC': 0.9461,
 'num_users': 6040,
 'num_interactions': 791718}

In [ ]:
test_metrics1 = evaluate( kernelized_rr, data, item_propensity, sample_user, test_set_eval = True)

In [ ]:
test_metrics1

{'HR@10': 16.9743,
 'HR@100': 45.9048,
 'NDCG@10': 16.0648,
 'NDCG@100': 27.4142,
 'PSP@10': 1.8038,
 'PSP@100': 4.9726,
 'AUC': 0.923,
 'num_users': 6040,
 'num_interactions': 1504728}

Distiil-cf HR@10 = 20.1002 | HR@100 = 47.2420 | NDCG@10 = 17.8793 | NDCG@100 = 28.1907 | PSP@10 = 2.0012 | PSP@100 = 5.2494 | AUC = 0.9211 | num_users = 500.0000 (VAL)